In [1]:
from sepal_ui import widgetFactory as wf
from sepal_ui import widgetBinding as wb
from sepal_ui.scripts import utils as su

from utils import messages as ms
import ipyvuetify as v
from functools import partial
from scripts import export
from sepal_ui.scripts import mapping
import os

In [2]:
#mutable object 
class Export_io:
    def __init__(self):
        #input
        self.vue = None
        
export_io = Export_io()

In [3]:
#validation of inputs
export_valid_output = wf.OutputWidget(ms.VALID_DATA)
export_valid_btn = wf.ProcessBtn(ms.VALID_BTN)
id_ = 'export_widget'
title = "Validate the input data"

export_valid = wf.Tile(
    id_, 
    title, 
    inputs = [''], 
    btn=export_valid_btn, 
    output=export_valid_output
)

In [4]:
#result tile 
title="Export results"

export_results = wf.Tile(id_, title, inputs=[''])

In [5]:
#export data
export_output = wf.OutputWidget(ms.EXPORT_INTRO)
export_btn = wf.ProcessBtn(ms.EXPORT_BTN)
title = "Export data"

export_tile = wf.Tile(
    id_, 
    title, 
    btn=export_btn, 
    output=export_output,
    inputs=['']
)

In [6]:
#only useful for debug 
import pandas as pd

In [7]:
#force the user to check tu data h's about to export 
def validate_data(widget, event, data, output):
    
    #toggle the loading button
    su.toggleLoading(widget)
    
    #load the input 
    #file = os.path.expanduser('~') + '/test_pts.txt'
    #file = os.path.expanduser('~') + '/sample_csvPilot.txt'
    file = os.path.expanduser('~') + '/sample_csvPilot_header.txt'
    pts = pd.read_csv(file)
    pts = pts.rename(columns={'LAT': 'lat', 'LON': 'lng', 'PLOTID': 'id'})
    
    #bands = 'Red, Green, Blue'
    #bands = 'Nir, Red, Green'
    bands = 'Nir, Swir1, Red'
    sources = ['landsat']
    #file = getattr(tb_io, 'file')
    #bands = getattr(viz_io, 'bands')
    #sources = getattr(viz_io, 'sources')
    
    #check input
    if not wb.checkInput(file, output, ms.MISSING_IN): return su.toggleLoading(widget)
    if not wb.checkInput(bands, output, ms.MISSING_IN): return su.toggleLoading(widget)
    if not wb.checkInput(sources, output, ms.MISSING_IN): return su.toggleLoading(widget)
    
    msg = ms.EXPORT_INPUTS.format(file, ' & '.join(sources), bands)
    
    su.displayIO(output, msg, 'success')
    
    #change attribute 
    setattr(export_io, 'vue', True)
    
    #toggle the loading button
    su.toggleLoading(widget)
    
    return 

export_valid_btn.on_event('click', partial(
    validate_data,
    output=export_valid_output
))
    

In [8]:
from scripts import export

#export data 
def start_process(widget, event, data, output, result):
    
    #toggle the loading button
    su.toggleLoading(widget)
     
    #load the input 
    #file = os.path.expanduser('~') + '/test_pts.txt'
    #file = os.path.expanduser('~') + '/sample_csvPilot.txt'
    file = os.path.expanduser('~') + '/sample_csvPilot_header.txt'
    pts = pd.read_csv(file)
    pts = pts.rename(columns={'LAT': 'lat', 'LON': 'lng', 'PLOTID': 'id'})
    
    #bands = 'Red, Green, Blue'
    #bands = 'Nir, Red, Green'
    bands = 'Nir, Swir1, Red'
    sources = ['landsat']
    #file = getattr(tb_io, 'file')
    #bands = getattr(viz_io, 'bands')
    #sources = getattr(viz_io, 'sources')
    
    #check the validation 
    valid = getattr(export_io, 'vue')
    
    #check only validation 
    if not wb.checkInput(valid, output, ms.NO_VALID): return su.toggleLoading(widget)
    
    #start the exporting process 
    pdf_file = export.run(file, pts, bands, sources, output)
    
    #add a download button 
    dwn = wf.DownloadBtn('PDF time series', path=pdf_file)
    result.children[0].children = [result.children[0].children[0], dwn]
    
    #toggle the loading button
    su.toggleLoading(widget)
    
    return 

export_btn.on_event('click', partial(
    start_process,
    output=export_output,
    result = export_results
))
    

In [9]:
export_valid

Layout(align_center=True, children=[Card(children=[Html(children=['Validate the input data'], tag='h2'), Flex(…

In [10]:
export_tile

Layout(align_center=True, children=[Card(children=[Html(children=['Export data'], tag='h2'), Flex(children=[La…

In [11]:
export_results

Layout(align_center=True, children=[Card(children=[Html(children=['Export results'], tag='h2'), Flex(children=…